# Figuring out the pixel mapping

In [ ]:
from dasbus.connection import SystemMessageBus
import numpy as np

bus = SystemMessageBus()

proxy = bus.get_proxy(
    "org.asuslinux.Daemon",
    "/org/asuslinux/Anime"
)

In [ ]:
bytes = np.array([0x00 for i in range(0, 1254)])
def write(start, end, val=0xff):
    """Glows all the LEDs between [`start`, `end`) index on the anime matrix (includes start index but not the end index)."""
    # assert end == -1 or start <= end
    b = bytes.copy()
    b[start:end] = val
    print(f'{(len(b[start:end]))} elements changed between {start} and {end}')

    # proxy.Write(b)
    proxy.Write((b, 0))  # new format

In [ ]:
# Go through each row and establish all the live as well as dummy LEDs in each row.
start = 2
end = 33

write(start, end)

In [ ]:
# The following variables were obtained by trial and error on the previous cell
dead_pixels = [
    0, 33,  # row 0
    67, 68, # row 1
    135, 136,
    203, 
    333,
    395,
    455,
    513,
    569,
    623,
    675,
    725,
    773,
    819,
    863,
    905,
    945,
    983,
    1019,
    1053,
    1085,
    1115,
    1143,
    1169,
    1193,
    1215,
    1235,
    1245,
    1246,
    1247,
    1248,
    1249, 
    1250,
    1251,
    1252,
    1253,
    ]
row0 = slice(1, 33)
row1 = slice(34, 67)
row2 = slice(69, 102)
row3 = slice(102, 135)
row4 = slice(137, 170)
row5 = slice(170, 203)
row6 = slice(204, 237)
row7 = slice(237, 270)
row8 = slice(270, 302)
row9 = slice(302, 333)
row10 = slice(334, 365)
row11 = slice(365, 395)
row12 = slice(396, 426)
row13 = slice(426, 455)
row14 = slice(456, 485)
row15 = slice(485, 513)
row16 = slice(514, 542)
row17 = slice(542, 569)
row18 = slice(570, 597)
row19 = slice(597, 623)
row20 = slice(624, 650)
row21 = slice(650, 675)
row22 = slice(676, 701)
row23 = slice(701, 725)
row24 = slice(726, 750)
row25 = slice(750, 773)
row26 = slice(774, 797)
row27 = slice(797, 819)
row28 = slice(820, 842)
row29 = slice(842, 863)
row30 = slice(864, 885)
row31 = slice(885, 905)
row32 = slice(906, 926)
row33 = slice(926, 945)
row34 = slice(946, 965)
row35 = slice(965, 983)
row36 = slice(984, 1002)
row37 = slice(1002, 1019)
row38 = slice(1020, 1037)
row39 = slice(1037, 1053)
row40 = slice(1054, 1070)
row41 = slice(1070, 1085)
row42 = slice(1086, 1101)
row43 = slice(1101, 1115)
row44 = slice(1116, 1130)
row45 = slice(1130, 1143)
row46 = slice(1144, 1157)
row47 = slice(1157, 1169)
row48 = slice(1170, 1182)
row49 = slice(1182, 1193)
row50 = slice(1194, 1205)
row51 = slice(1205, 1215)
row52 = slice(1216, 1226)
row53 = slice(1226, 1235)
row54 = slice(1236, 1245)
rows = [row0, row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, row12, row13, row14, row15, row16, row17, row18, row19, row20, row21, row22, row23, row24, row25, row26, row27, row28, row29, row30, row31, row32, row33, row34, row35, row36, row37, row38, row39, row40, row41, row42, row43, row44, row45, row46, row47, row48, row49, row50, row51, row52, row53, row54]

In [ ]:
# Sanity check to ensure we covered all the pixels
idxs = np.zeros(1254)  # The length of the array needs to be 1254 exactly

for row in [row0, row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, row12, row13, row14, row15, row16, row17, row18, row19, row20, row21, row22, row23, row24, row25, row26, row27, row28, row29, row30, row31, row32, row33, row34, row35, row36, row37, row38, row39, row40, row41, row42, row43, row44, row45, row46, row47, row48, row49, row50, row51, row52, row53, row54]:
    idxs[row] = 1

for dead_pixel in dead_pixels:
    assert idxs[dead_pixel] == 0

assert sum(idxs) + len(dead_pixels) == 1254
assert set(np.arange(len(idxs))[idxs==0]).difference(set(dead_pixels)) == set()

In [ ]:
# Length of all the rows
row_lens = [len(range(1254)[row]) for row in [row0, row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, row12, row13, row14, row15, row16, row17, row18, row19, row20, row21, row22, row23, row24, row25, row26, row27, row28, row29, row30, row31, row32, row33, row34, row35, row36, row37, row38, row39, row40, row41, row42, row43, row44, row45, row46, row47, row48, row49, row50, row51, row52, row53, row54]]

In [ ]:
# Handling the 'dead' pixels or the 'dummy' LEDs 
dead_rows = [0]
for i in range(1, len(rows)):
    if rows[i-1].stop != rows[i].start:
        dead_rows.append((i, rows[i].start))


pxls = dead_pixels.copy()
dead_indices = [(0, 0), (0, -1)]  # 0th row is special
pxls.remove(0)  # Remove the dead pixels associated with the 0th row
pxls.remove(33)

# Convert dead pixels to their indices on the image
for row_num, start_idx in dead_rows[1:]:
    dead_pos = [pxls[i] for i in range(len(pxls)) if pxls[i] < start_idx]
    dead_indices.append((row_num, len(dead_pos)))
    [pxls.remove(item) for item in dead_pos]
    # print(row_num, start_idx, dead_pos)  #DEBUG

In [ ]:
# Map the LED locations
import matplotlib.pyplot as plt

# Making it 56x35 to accomodate all the live, and dummy pixels in the same image. The actual area covered by live LEDs is slightly smaller
img = np.zeros((56, 35), dtype=np.uint8) 
img[0, 2:-1] += 1  # 0th row is special

# Actual LEDs
for i in range(1, len(row_lens)):
    img[i, -row_lens[i]:] += 1

# Dead pixels
img[0, -1] += 2
img[0, -row_lens[0]-2] += 2
for row_num, num_dead in dead_indices[2:]:
    for i in range(num_dead):
        img[row_num, -row_lens[row_num]-i-1] += 2

# Extra dead pixels at the very end
img[-1, -np.arange(len(pxls)) - 1] += 2

# Sanity Checks
assert np.sum(img==2) == len(dead_pixels)
assert np.sum(img==1) == sum(idxs)
assert np.sum(img!=0) == 1254

plt.imshow(img)


In [ ]:
# Make sure we covered all the pixels and did not overlap any two different types
leds_mask = np.zeros((56, 35, 3), dtype=bool)
leds_mask[..., 0] = img == 1
leds_mask[..., 1] = img == 2
leds_mask[..., 2] = img != 0

# Save the pixel map

In [ ]:
np.save("led_mask_GA402.npy", leds_mask)

# Load the pixel map and test display

In [ ]:
import numpy as np
from dasbus.connection import SystemMessageBus
import numpy as np
import time

bus = SystemMessageBus()


proxy = bus.get_proxy(
    "org.asuslinux.Daemon",
    "/org/asuslinux/Anime"
)

led_mask = np.load('led_mask_GA401.npy')

def image_to_anime(img, led_mask):
    # We only need values in these locations
    mask_all_leds = led_mask[..., -1]

    # Make sure the image is in the right format and shape
    assert img.dtype == np.uint8
    assert img.shape == mask_all_leds.shape

    # Convert to anime-matrix bytes format
    anime_bytes = []
    for i in range(mask_all_leds.shape[0]):
        img_row, mask_row = img[i], mask_all_leds[i]
        anime_bytes.extend([img_pixel for img_pixel, mask_pixel in zip(img_row, mask_row) if mask_pixel])

    # Write to the display
    # proxy.Write(anime_bytes)
    # proxy.Write((anime_bytes, 0))  # new format
    return anime_bytes

### A very hasty test

In [ ]:
# Should make all the LEDs change gradually from black to white
img = np.ones((56, 35), dtype=np.uint8)
while True:
    try:
        for i in range(0,255, 50):
            time.sleep(0.0001)
            # proxy.Write(image_to_anime((img * i).astype(np.uint8), led_mask))
            proxy.Write((image_to_anime((img * i).astype(np.uint8), led_mask), 0))  # new format
    except KeyboardInterrupt:
        break